# Sistemas de Big Data - Apartado 4

Volvemos a trabajar con el conjunto de datos de pingüinos y ahora se nos pide aplicar un modelo de clasificación (la variable objetivo es la especie del pingüino). Puedes usar cualquiera de los que se implementan en MLlib (en los apuntes hemos visto el de regresión logística y el de random forests). Utiliza el 80% del conjunto de datos para entrenar el modelo y el 20% restante para obtener las predicciones y evaluar el modelo. Debes obtener cuál es la exactitud (accuracy) y la matriz de confusión, para observar qué tan bueno ha sido el resultado. Comenta los resultados obtenidos.

Debes implementarlo mediante un cuaderno de Google Colab, utilizando PySpark y la librería MLlib.

In [2]:
import pyspark
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import when, col, isnan, isnull
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator
from pyspark.sql.types import DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

spark = SparkSession.builder \
    .appName("Penguins_clustering") \
    .getOrCreate()

url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/penguins_size.csv"

!wget -q $url -O penguins_size.csv

penguins_df = spark.read.csv("penguins_size.csv", header=True, inferSchema=True)

In [3]:
# Limpiamos el CSV
for column in penguins_df.columns:
    penguins_df = penguins_df.withColumn(
        column,
        when(
            (col(column) == "") | (col(column) == "NA") | (col(column) == "null"),
            None
        ).otherwise(col(column))
    )

penguins_clean = penguins_df.na.drop()

# Formateamos los datos
penguins_clean = penguins_clean.withColumn("culmen_length_mm", penguins_clean["culmen_length_mm"].cast(DoubleType()))
penguins_clean = penguins_clean.withColumn("culmen_depth_mm", penguins_clean["culmen_depth_mm"].cast(DoubleType()))
penguins_clean = penguins_clean.withColumn("flipper_length_mm", penguins_clean["flipper_length_mm"].cast(DoubleType()))
penguins_clean = penguins_clean.withColumn("body_mass_g", penguins_clean["body_mass_g"].cast(DoubleType()))

print(f"\nFilas originales: {penguins_df.count()}")
print(f"Filas después de elimiar NA's: {penguins_clean.count()}")

print("\nDistribución:")
penguins_clean.groupBy("species").count().orderBy("count", ascending=False).show()


Filas originales: 344
Filas después de elimiar NA's: 334

Distribución:
+---------+-----+
|  species|count|
+---------+-----+
|   Adelie|  146|
|   Gentoo|  120|
|Chinstrap|   68|
+---------+-----+



In [18]:
species_indexer = StringIndexer(inputCol="species", outputCol="label").fit(penguins_clean)
penguin_indexed = species_indexer.transform(penguins_clean)

categorical_cols = ["island", "sex"]

indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index")
    for col in categorical_cols
]

encoders = [
    OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_vec")
    for col in categorical_cols
]

numeric_cols = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"]

assembler_inputs = numeric_cols + [f"{col}_vec" for col in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

pipeline_stages = indexers + encoders + [assembler]
pipeline = Pipeline(stages=pipeline_stages)

In [32]:
# Variable objetivo (species)
species_indexer = StringIndexer(inputCol="species", outputCol="label").fit(penguins_clean)
penguin_indexed = species_indexer.transform(penguins_clean)

print("\nRelación especie - etiqueta:")
species_mapping = {float(i): label for i, label in enumerate(species_indexer.labels)}
for index, species in species_mapping.items():
    print(f"Etiqueta {index} - {species}")

categorical_cols = ["island", "sex"]

indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index")
    for col in categorical_cols
]

encoders = [
    OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_vec")
    for col in categorical_cols
]

numeric_cols = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"]

assembler_inputs = numeric_cols + [f"{col}_vec" for col in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

pipeline_stages = indexers + encoders + [assembler]
pipeline = Pipeline(stages=pipeline_stages)


Relación especie - etiqueta:
Etiqueta 0.0 - Adelie
Etiqueta 1.0 - Gentoo
Etiqueta 2.0 - Chinstrap


Para preparar los datos, primero he convertido la variable objetivo a índices numéricos. También he transformado las variables categóricas con OneHotEncoder, y he usado un VectorAssembler para juntar todas las características (numéricas y categóricas) en un único vector. Todo esto lo he hecho usando un pipeline para dejarlo bien ordenado.

In [29]:
# Dividimos el dataset en test (20%) y entrenamiento (80%)
train_data, test_data = penguin_indexed.randomSplit([0.8, 0.2], seed=42)
print(f"\nNúmero de muestras de entrenamiento (80%): {train_data.count()}")
print(f"Número de muestras de test (20%): {test_data.count()}")

pipeline_model = pipeline.fit(train_data)
train_data_transformed = pipeline_model.transform(train_data)
test_data_transformed = pipeline_model.transform(test_data)

print("\nDatos transformados:")
train_data_transformed.select("label", "features").show(5, truncate=False)


Número de muestras de entrenamiento (80%): 284
Número de muestras de test (20%): 50

Datos transformados:
+-----+----------------------------------------+
|label|features                                |
+-----+----------------------------------------+
|0.0  |[34.5,18.1,187.0,2900.0,1.0,0.0,0.0,1.0]|
|0.0  |[35.0,17.9,190.0,3450.0,1.0,0.0,0.0,1.0]|
|0.0  |[35.3,18.9,187.0,3800.0,1.0,0.0,0.0,1.0]|
|0.0  |[35.5,16.2,195.0,3350.0,1.0,0.0,0.0,1.0]|
|0.0  |[35.7,16.9,185.0,3150.0,1.0,0.0,0.0,1.0]|
+-----+----------------------------------------+
only showing top 5 rows



In [33]:
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10, regParam=0.1)
lr_model = lr.fit(train_data_transformed)
lr_predictions = lr_model.transform(test_data_transformed)

print("Pequeña muestra de predicciones (Regresión Logística):")
lr_predictions.select("label", "prediction", "probability").show(5)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"\nPrecisión del modelo de Regresión Logística: {lr_accuracy:.4f}")

lr_pred_pd = lr_predictions.select("label", "prediction").toPandas()
conf_matrix_lr = pd.crosstab(
    lr_pred_pd["label"],
    lr_pred_pd["prediction"],
    rownames=["Actual"],
    colnames=["Predición"]
)

conf_matrix_lr.index = [species_mapping[i] for i in conf_matrix_lr.index]
conf_matrix_lr.columns = [species_mapping[i] for i in conf_matrix_lr.columns]

print("\nMatriz de confusión (Regressió Logística):")
print(conf_matrix_lr)

Pequeña muestra de predicciones (Regresión Logística):
+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.91487924349481...|
|  0.0|       0.0|[0.93834666075675...|
|  0.0|       0.0|[0.93132695308119...|
|  0.0|       0.0|[0.94624054660143...|
|  0.0|       0.0|[0.94609557066992...|
+-----+----------+--------------------+
only showing top 5 rows


Precisión del modelo de Regresión Logística: 1.0000

Matriz de confusión (Regressió Logística):
           Adelie  Gentoo  Chinstrap
Adelie         24       0          0
Gentoo          0      17          0
Chinstrap       0       0          9


In [34]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=5,
    seed=42
)

rf_model = rf.fit(train_data_transformed)

rf_predictions = rf_model.transform(test_data_transformed)

print("Pequeña muestra de predicciones (Random Forest):")
rf_predictions.select("label", "prediction", "probability").show(5)

rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"\nPrecisión del modelo Random Forest: {rf_accuracy:.4f}")

rf_pred_pd = rf_predictions.select("label", "prediction").toPandas()
conf_matrix_rf = pd.crosstab(
    rf_pred_pd["label"],
    rf_pred_pd["prediction"],
    rownames=["Actual"],
    colnames=["Predición"]
)

conf_matrix_rf.index = [species_mapping[i] for i in conf_matrix_rf.index]
conf_matrix_rf.columns = [species_mapping[i] for i in conf_matrix_rf.columns]

print("\nMatriz de confusión (Random Forest):")
print(conf_matrix_rf)

Pequeña muestra de predicciones (Random Forest):
+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.99370481254892...|
|  0.0|       0.0|[0.99477070402179...|
|  0.0|       0.0|[0.96780746470233...|
|  0.0|       0.0|[0.97453814588225...|
|  0.0|       0.0|[0.99524120250549...|
+-----+----------+--------------------+
only showing top 5 rows


Precisión del modelo Random Forest: 1.0000

Matriz de confusión (Random Forest):
           Adelie  Gentoo  Chinstrap
Adelie         24       0          0
Gentoo          0      17          0
Chinstrap       0       0          9


## Conclusiones
Después de preparar y limpiar los datos de pingüinos, probé dos modelos de clasificación para predecir la especie a partir de características físicas y categóricas: regresión logística y random forest.

Ambos modelos se comportaron sorprendentemente bien, los dos alcanzaron una precisión del 100 % en el conjunto de test. Las matrices de confusión mostraron que no hubo ni un solo error de clasificación. Cada pingüino fue clasificado correctamente en su especie, lo cual me pareció bastante curioso. No es muy habitual obtener resultados tan perfectos, así que me hace pensar que este dataset está bastante limpio y que las clases están muy bien separadas.

Una cosa que me llamó la atención es que incluso la regresión logística, que es un modelo bastante sencillo, funcionó igual de bien que el random forest, que en teoría es más potente. En este caso parece que no hace falta un modelo complejo para obtener buenos resultados.

En resumen, fue un experimento bastante redondo. Aunque los resultados fueron excelentes, también me dejan con la duda de si el modelo se comportaría igual de bien con datos nuevos o en un contexto más real. Tal vez lo siguiente sería probar con validación cruzada o introducir algo de ruido para ver cómo aguantan los modelos.